In [2]:
import os

# change the working directory to the location of files
os.chdir('D:/eo_data/Ibague/')
print(os.getcwd())
# store the files list to a variable
eo_files = os.listdir(os.getcwd())
print(eo_files)

D:\eo_data\Ibague
['data', 'S1_GRD', 'S2_L1C']


In [48]:
# Unzip files
import zipfile

# Check if a data folder exist
#if not os.path.exists('data'):
#    os.makedirs('data')
#    print 'data folder' + ' was created'
# Check if the name of the data folder is in
#if 'data' in eo_files:
#    eo_files.remove('data')
##TODO catch error when a directory is in eo_files

def unzipfiles(path):
    """
    Unzips every zipfile in the path, and stores in same directory
    Args:
        path (string): string of directory where zipfiles are located 
    """
    for im_id in os.listdir(path):
        if not os.path.exists('data/'+im_id[:-3]+'SAFE'):
            print('Unzipping ' + im_id)
            zip_ref = zipfile.ZipFile(path+im_id, 'r')
            zip_ref.extractall('data')
            zip_ref.close()
        else:
            print(im_id[:-4] + ' was already uncompressed')

In [4]:
import zipfile

zip_ref = zipfile.ZipFile('/home/azalazar/data/test_l2A.zip', 'r')
zip_ref.extractall('/home/azalazar/data/')
zip_ref.close()

## Process L1C to L2A images using sen2cor
Call in console. To-do: check integration

In [1]:
import os, re

## TODO: remove hard coding of sen2cor installation
def sen2cor_L2A (res, prod): # 
    """
    Function to call sen2cor L2A_Process for obtaining L2A products
    
    res (str/num): resolution, accepts 10, 20, 60, or all 
    prod (str): location of S1 L1C product
    """
    # Hard-code location of sen2cor installation
    os.chdir("/home/azalazar/DL_Temp/Sen2Cor-02.05.05-Linux64/bin")
    # Coerce resolution to string
    res = str(res)
    # Execute L2A_Process with resolution parameter when specified
    if res == 'all':
        os.system("bash L2A_Process" + " " + prod)
    else:
        os.system("bash L2A_Process --resolution=" + res + " " + str(prod))

def sen2cor_L2A_batch (res, L1Cdir):
    """
    Function to batch processing of S1-L1C files in a directory to S1-L2A
    
    res (str/num): resolution, accepts 10, 20, 60, or all
    L1Cdir (str): location of S1 L1C products
    """
    # Put S1 L1C directory names in list
    L1C_files = filter(re.compile(r'^S2.....L1C').search, os.listdir(L1Cdir))
    print("{} L1C files found in directory".format(str(len(L1C_files))))
    
    for L1C_file in L1C_files: # Iterate over directory names
        # Call sen2cor function for individual product
        print("Processing {}".format(L1C_file))
        sen2cor_L2A(res, L1Cdir+L1C_file)
        
## Based on:     
## Rodrigo Almeida, Maya Situnayake, Kees Baake, Mortimer Werther, Timon Weitkamp, Arnan Araza. Wageningen University.
## Academic Consultancy Project for EagleSensing. Remote Sensing and GIS Integration course, Period 6, 2016-2017.
## https://github.com/rodrigoalmeida94/ACT_EagleSensing.

In [12]:
data_dir = "/home/azalazar/data/"
print(os.getcwd())

#sen2cor_L2A_batch('all', server_dir)

/home/azalazar/GitHub/ciat_monitor_crops


## Pre-process L2A images

In [5]:
# import required packages
import snappy
import re

from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from snappy import ProductIO
from snappy import HashMap
import shutil
import os  
import ast

from snappy import GPF

GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
HashMap = snappy.jpy.get_type('java.util.HashMap')

In [6]:
# set location of products
data_dir = "/home/azalazar/data/"

# get geoJSON file with region extent
regName = "Ibague"
regPath = "/mnt/workspace_cluster_6/TRANSVERSAL_PROJECTS/MADR/COMPONENTE_2/Imagenes_Satelitales/Sentinel_2/" + \
                      "JSON_Ibague/IbagueJSON.geojson"
geom = geojson_to_wkt(read_geojson(regPath))

# get a list location of S2 L2A products

prdlist = filter(re.compile(r'^S2.....L2A').search, os.listdir(data_dir))
#prdlist = map(lambda x: data_dir+x, prdlist)

list(prdlist)

['S2A_MSIL2A_20171210T152631_N0206_R025_T18NVK_20171210T170838.SAFE']

In [10]:
os.chdir(data_dir)

In [15]:
## Pre-processing of Sentinel-2 L2A products

def preSentinel2(L2Alist, subregion, regname):
    """
    
    L2Alist (list): list of location of directory of L2A products (.SAFE dir)
    subregion (str): wkt string of extent of region of interest
    regname (str): string identifier of the region of interest
    """
    
    # Create a dictionary to read Sentinel-1 L1 GRD products
    product = {}
    for element in L2Alist:
        product[element[:-5]] = {}
        print(element)
    
    for key, value in product.iteritems():    
        # Read the product
        print('Reading {}'.format(key+'.SAFE/MTD_MSIL2A.xml'))
        value['GRD'] = ProductIO.readProduct(key+'.SAFE/MTD_MSIL2A.xml')
        
        # Resample all bands to 10m resolution
        resample_subset = HashMap()
        #resample_subset.put('targetResolution', 10)
        print('Resampling {}'.format(key))
        #value['res10'] = GPF.createProduct('Resample', resample_subset, value['GRD'])
        
        # Subset to area of interest
        param_subset = HashMap()
        param_subset.put('geoRegion', subregion)
        param_subset.put('outputImageScaleInDb', False)
        param_subset.put('sourceBandNames', 'B2, B3, B4, B8, B11, B12,\
        quality_cloud_confidence,quality_scene_classification')
        print('Subsetting {}'.format(key))
        #value['sub'] = GPF.createProduct("Subset", param_subset, value['res10'])
        
        # Write product
        print('Writing {} subset resampled to 10m'.format(key))
        ProductIO.writeProduct(value['GRD'], key+'_'+regname+'_subset', 'BEAM-DIMAP')
        
        # Dispose all the intermediate products
        value['GRD'].dispose()
        #value['res10'].dispose()
        #value['sub'].dispose()
        

In [16]:
preSentinel2(prdlist, geom, regName)

# ambiguous Java method call, too many matching method overloads found
# todo test if data is corrupted or there is an installation problem

S2A_MSIL2A_20171210T152631_N0206_R025_T18NVK_20171210T170838.SAFE
Reading S2A_MSIL2A_20171210T152631_N0206_R025_T18NVK_20171210T170838.SAFE/MTD_MSIL2A.xml
Resampling S2A_MSIL2A_20171210T152631_N0206_R025_T18NVK_20171210T170838
Subsetting S2A_MSIL2A_20171210T152631_N0206_R025_T18NVK_20171210T170838
Writing S2A_MSIL2A_20171210T152631_N0206_R025_T18NVK_20171210T170838 subset resampled to 10m


RuntimeError: java.lang.IllegalArgumentException: [product] is null

In [4]:
from snappy import ProductIO, GPF, jpy

GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
HashMap = jpy.get_type('java.util.HashMap')

#collection of preprocessed files
s1 = 'D:/eo_data/Ibague/data/prep/VH_specklefil_dB.dim'
s2 = 'D:/eo_data/Ibague/data/prep/VV_specklefil_dB.dim'
master = 'D:/eo_data/Ibague/data/test.data/B1.img'

processed_files = [master,s1,s2]

product_set=[]

for f in processed_files:
    product_set.append(ProductIO.readProduct(f))
    print("Reading "+f)
      
#define the stack parameters
params = HashMap()
params.put('resamplingType', 'NEAREST_NEIGHBOUR')
params.put('initialOffsetMethod', 'Product Geolocation')
params.put('extent', 'Master')

#make the stack
print("Creating stack...")
create_stack = GPF.createProduct('CreateStack', params, product_set)

#write the stack
print("Saving resulting product...")
ProductIO.writeProduct(create_stack, 'D:/eo_data/Ibague/data/create_stack', 'BEAM-DIMAP')
print("Done.")

Reading D:/eo_data/Ibague/data/test.data/B1.img
Reading D:/eo_data/Ibague/data/prep/VH_specklefil_dB.dim
Reading D:/eo_data/Ibague/data/prep/VV_specklefil_dB.dim
Creating stack...
Saving resulting product...
Done.
